In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%matplotlib inline
import networkx as nx 
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt 

N=2;S0=1

r=0.75;a=-0.5;b=2

p = (r-a)/(b-a)
q = (b-r)/(b-a)

def plot_tree(g):
    plt.figure(figsize=(20,10))
    pos={}
    lab={}
    
    for n in g.nodes():
        pos[n]=(n[0],n[1])
        lab[n]=float("{0:.2f}".format(g.node[n]['value']))
    
    elarge=g.edges(data=True)
    nx.draw_networkx_edges(g,pos,edgelist=elarge)
    nx.draw_networkx_labels(g,pos,lab,font_size=15)
    plt.ylim(-N+0.5,N+1.5) 
    plt.xlim(-0.5,N+0.5)
    plt.show()
    
def graph_stock():
    S=nx.Graph()
    for k in range(0,N):
        for l in range(-k+1,k+3,2):
            S.add_edge((k,l),(k+1,l+1))
            S.add_edge((k,l),(k+1,l-1))
            
    for n in S.nodes():        
        k=n[0]
        l=n[1]-1
        S.node[n]['value']=S0*((1.0+b)**((k+l)/2))*((1.0+a)**((k-l)/2))
    return S

plot_tree(graph_stock())

In [ ]:
def European_call_hedge_risky(K):

    price = nx.Graph()
    hedge_risky = nx.Graph()
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                price.add_edge((k,l),(k+1,l+1))
                price.add_edge((k,l),(k+1,l-1))
                hedge_risky.add_edge((k,l),(k+1,l+1))
                hedge_risky.add_edge((k,l),(k+1,l-1))
    
    for l in range(-N+1,N+3,2):
        price.node[(N,l)]['value'] = np.maximum(S.node[(N,l)]['value']-K,0)
        hedge_risky.node[(N,l)]['value'] = 0

    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            price.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']*p+price.node[(k+1,l-1)]['value']*q)/(1+r)
            hedge_risky.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']-price.node[(k+1,l-1)]['value'])/(b-a)/(S.node[(k,l)]['value'])
    return hedge_risky

In [ ]:
def European_call_hedge_riskless(K):

    price = nx.Graph()
    hedge_riskless = nx.Graph()
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                price.add_edge((k,l),(k+1,l+1))
                price.add_edge((k,l),(k+1,l-1))
                hedge_riskless.add_edge((k,l),(k+1,l+1))
                hedge_riskless.add_edge((k,l),(k+1,l-1))
    
    for l in range(-N+1,N+3,2):
        price.node[(N,l)]['value'] = np.maximum(S.node[(N,l)]['value']-K,0)
        hedge_riskless.node[(N,l)]['value'] = 0

    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            price.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']*p+price.node[(k+1,l-1)]['value']*q)/(1+r)
            hedge_riskless.node[(k,l)]['value'] = ((1+b)*price.node[(k+1,l-1)]['value']-(1+a)*price.node[(k+1,l+1)]['value'])/(b-a)/pow(1+r,k+1)
    return hedge_riskless

In [ ]:
K = input("Strike K=")

print('Underlying asset prices:')
plot_tree(graph_stock())
print('Risky hedging strategy:')
plot_tree(European_call_hedge_risky(float(K)))
print('Riskless hedging strategy:')
plot_tree(European_call_hedge_riskless(float(K)))

In [ ]:
def Hedge_then_price(K):

    hedge_riskless = European_call_hedge_riskless(K)
    hedge_risky = European_call_hedge_risky(K)
    S = graph_stock()
    hedge_then_price = nx.Graph()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                hedge_then_price.add_edge((k,l),(k+1,l+1))
                hedge_then_price.add_edge((k,l),(k+1,l-1))
        
    for l in range(-N+1,N+3,2):
        hedge_risky.node[(N,l)]['value'] = 0
        hedge_then_price.node[(N,l)]['value'] = 0

    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            hedge_then_price.node[(k,l)]['value'] = hedge_risky.node[(k,l)]['value']*S.node[(k,l)]['value']+hedge_riskless.node[(k,l)]['value']*(1+r)**k
    return hedge_then_price

In [ ]:
K = input("Strike K=")

print('Underlying asset prices:')
plot_tree(graph_stock())
print('Risky hedging strategy:')
plot_tree(European_call_hedge_risky(float(K)))
print('Riskless hedging strategy:')
plot_tree(European_call_hedge_riskless(float(K)))
print('Hedge then price:')
plot_tree(Hedge_then_price(float(K)))